In [ ]:
#Change this for to create your folder and save experiments
model_name = 'komal/gpt2-medium-classifier_finetuning_24layer'

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

# Define the persistent path on your Drive
# IMPORTANT: This folder must exist in your Google Drive!
PERSISTENT_BASE_DIR = f'/content/drive/MyDrive/Efficient_AI_Project/EarlyExit_Experiments/{model_name}'
os.makedirs(PERSISTENT_BASE_DIR, exist_ok=True)

Mounted at /content/drive


In [ ]:
import sys

!git clone https://github.com/komalniraula/adaptive-inference-llm

repo_name = 'adaptive-inference-llm' # Must match the folder created by git clone
project_path = os.path.join('/content', repo_name)

# Append the project root directory to the system path

sys.path.append(project_path)

Cloning into 'adaptive-inference-llm'...
remote: Enumerating objects: 281, done.
remote: Counting objects: 100% (281/281), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 281 (delta 137), reused 213 (delta 71), pack-reused 0 (from 0)
Receiving objects: 100% (281/281), 6.71 MiB | 35.81 MiB/s, done.
Resolving deltas: 100% (137/137), done.


In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
import random
import itertools
import json
import time
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer

from evaluation.dataset_loaders.sst2 import load_sst2

In [ ]:
# -----------------------------
# Load datasets
# -----------------------------
# Load the labeled SST-2 train split (67k samples)
sst2_train_data = load_sst2(task='train', fraction=1)

# Load the SST-2 validation split (872 samples) for testing
# This is equivalent to your previous sst2_test = ds["validation"]
sst2_test_data = load_sst2(task='test', fraction=1)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [ ]:
# -----------------------------
# Convert datasets to (text,label)
# -----------------------------
# The external load_sst2 function should already return a list/Dataset
# where elements can be converted.
# We adapt the conversion based on the expected output of your loader module.

def convert_sst2(sample):
    # Assuming your module returns samples with 'text' and 'label' keys
    # based on the preprocess function in your original loader.
    if "text" in sample:
        return sample["text"], int(sample["label"])
    elif "sentence" in sample:
        # Fallback for the raw SST-2 key
        return sample["sentence"], int(sample["label"])
    else:
         raise ValueError("SST-2 sample missing expected keys for conversion.")

test_pairs  = [convert_sst2(x) for x in sst2_test_data]

print(f"Total test samples: {len(test_pairs)}")


# -----------------------------
# Dataset class
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

class SentimentDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text, label = self.data[idx]
        enc = self.tokenizer(
            text,
            add_special_tokens=False,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }


# -----------------------------
# Dynamic padding collate_fn
# -----------------------------
def collate_fn(batch):
    input_ids = [b["input_ids"] for b in batch]
    labels = torch.tensor([b["labels"] for b in batch])

    max_len = max(len(ids) for ids in input_ids)
    pad_id = tokenizer.pad_token_id

    padded = torch.full((len(batch), max_len), pad_id)
    for i, ids in enumerate(input_ids):
        padded[i, :len(ids)] = ids

    attention_mask = (padded != tokenizer.eos_token_id).float()


    return {
        "input_ids": padded,
        "attention_mask": attention_mask,
        "labels": labels
    }

Total test samples: 872


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
test_ds = SentimentDataset(test_pairs, tokenizer)

test_loader = DataLoader(
    test_ds,
    batch_size=1,
    shuffle=False,
    collate_fn=collate_fn
)

print("Test loader batches:", len(test_loader))

# Optional: Inspect first batch
example = next(iter(test_loader))
print(example["input_ids"].shape)
print(example["attention_mask"].shape)
print(example["labels"])

Test loader batches: 872
torch.Size([1, 27])
torch.Size([1, 27])
tensor([0])


In [ ]:
class GPT2EarlyExitClassifier(nn.Module):
    def __init__(self, model_name, exit_layers, num_labels=2):
        super().__init__()

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            return_dict=True
        )

        self.exit_layers = sorted(exit_layers)
        self.num_labels = num_labels

        hidden_size = self.model.config.hidden_size

        # ⚠️ MUST match training exactly
        self.exit_heads = nn.ModuleDict()
        for layer in self.exit_layers:
            self.exit_heads[str(layer)] = nn.Sequential(
                nn.Dropout(0.0),          # dropout inactive in eval()
                nn.Linear(hidden_size, self.num_labels)
            )

    @torch.no_grad()
    def early_exit_forward(self, input_ids, attention_mask, threshold):
        """
        TRUE adaptive inference:
        - executes GPT-2 layer-by-layer
        - exits immediately once confidence ≥ threshold
        """

        transformer = self.model.transformer
        exit_set = set(self.exit_layers)
        max_layer = self.exit_layers[-1]

        # ---- embeddings (HF-consistent) ----
        position_ids = attention_mask.long().cumsum(-1) - 1
        position_ids.masked_fill_(attention_mask == 0, 0)

        hidden = transformer.wte(input_ids) + transformer.wpe(position_ids)

        # ---- layer-wise execution ----
        for layer_idx, block in enumerate(transformer.h, start=1):
            hidden = block(hidden, attention_mask=attention_mask)[0]

            if layer_idx in exit_set:
                cls_vec = hidden[:, -1, :]
                logits = self.exit_heads[str(layer_idx)](cls_vec)
                probs = torch.softmax(logits, dim=-1)
                conf, pred = probs.max(dim=-1)

                if conf.item() >= threshold or layer_idx == max_layer:
                    return {
                        "prediction": pred.item(),
                        "confidence": conf.item(),
                        "exit_layer": layer_idx,
                        "logits": logits
                    }

        # safety fallback
        return {
            "prediction": pred.item(),
            "confidence": conf.item(),
            "exit_layer": max_layer,
            "logits": logits
        }

In [ ]:
@torch.no_grad()
def adaptive_early_exit_eval(
    model_path,
    exit_layers,
    data_loader,
    device,
    thresholds,
):
    model = GPT2EarlyExitClassifier(
        model_name="gpt2-medium",
        exit_layers=exit_layers,
        num_labels=2
    ).to(device)

    state_dict = torch.load(
        os.path.join(model_path, "pytorch_model.bin"),
        map_location=device
    )
    model.load_state_dict(state_dict)
    model.eval()

    results = {}

    for th in thresholds:
      correct = 0
      total = 0
      total_layers_used = 0
      total_tokens = 0
      start_time = time.time()

      for batch in data_loader:
          assert batch["input_ids"].size(0) == 1

          tokens = batch["attention_mask"].sum().item()
          total_tokens += tokens

          out = model.early_exit_forward(
              input_ids=batch["input_ids"].to(device),
              attention_mask=batch["attention_mask"].to(device),
              threshold=th
          )

          correct += int(out["prediction"] == batch["labels"].item())
          total_layers_used += out["exit_layer"]
          total += 1

      elapsed = time.time() - start_time

      results[f"threshold_{th}"] = {
          "accuracy": correct / total,
          "avg_layers_used": total_layers_used / total,
          "avg_latency_sec": elapsed / total,
          "tokens_per_sec": total_tokens / elapsed,
          "cost_saving_pct": 100 * (1 - (total_layers_used / total) / exit_layers[-1]),
          "num_samples": total
      }

    return results

In [ ]:
def evaluate_all_models(
    persistent_base_dir,
    exit_layers,
    test_loader,
    device,
    thresholds
):
    """
    Runs true adaptive early-exit inference on all models inside
    PERSISTENT_BASE_DIR and returns a sorted DataFrame.
    """

    all_rows = []

    for model_name in sorted(os.listdir(persistent_base_dir)):
        model_dir = os.path.join(persistent_base_dir, model_name)
        best_model_dir = os.path.join(model_dir, "best_model")

        if not os.path.isdir(best_model_dir):
            continue

        model_file = os.path.join(best_model_dir, "pytorch_model.bin")
        if not os.path.exists(model_file):
            continue

        print(f"\n🚀 Evaluating model: {model_name}")

        results = adaptive_early_exit_eval(
            model_path=best_model_dir,
            exit_layers=exit_layers,
            data_loader=test_loader,
            device=device,
            thresholds=thresholds,
        )

        for th in thresholds:
            r = results[f"threshold_{th}"]

            all_rows.append({
              "model_name": model_name,
              "threshold": th,
              "accuracy": r["accuracy"],
              "avg_layers_used": r["avg_layers_used"],
              "avg_latency_sec": r["avg_latency_sec"],
              "cost_saving_pct": r["cost_saving_pct"],
              "tokens_per_sec": r["tokens_per_sec"],
          })


    df = pd.DataFrame(all_rows)

    # Sort by latency (fastest first)
    df = df.sort_values(
        by="avg_latency_sec",
        ascending=True
    ).reset_index(drop=True)

    return df

In [ ]:
exit_layers = [3, 6, 9, 12, 15, 18, 21, 24]
THRESHOLDS = [0.5, 0.6, 0.7, 0.8, 0.9]
device = "cuda" if torch.cuda.is_available() else "cpu"

df_results = evaluate_all_models(
    persistent_base_dir=PERSISTENT_BASE_DIR,
    exit_layers=exit_layers,
    test_loader=test_loader,
    device = device,
    thresholds=THRESHOLDS
)


🚀 Evaluating model: lr1e-05_wd0.01_ep3_drop0.0_lossW0.9-0.8-0.7-0.6-0.5-0.4-0.3-0.2


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


🚀 Evaluating model: lr5e-05_wd0.01_ep3_drop0.0_lossW0.9-0.8-0.7-0.6-0.5-0.4-0.3-0.2


In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

df_results.to_csv('models')

In [ ]:
os.makedirs(PERSISTENT_BASE_DIR, exist_ok=True)

# Path
csv_path = os.path.join(
    PERSISTENT_BASE_DIR,
    "finetuned_model_early_exit_stats.csv"
)

# Save
df_results.to_csv(csv_path, index=False)

print(f"Saved df_results to: {csv_path}")

Saved df_results to: /content/drive/MyDrive/Efficient_AI_Project/EarlyExit_Experiments/komal/gpt2-medium-classifier_finetuning_24layer/finetuned_model_early_exit_stats.csv
